In [2]:
import psycopg2
from sqlalchemy import create_engine, Table, func
from sqlalchemy.orm import declarative_base, relationship, backref, sessionmaker

engine = create_engine('postgresql+psycopg2://pimat:pimat@db:5432/bakugan_db', echo=True)
Base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.types import Enum
import enum

class Side(enum.Enum):
    earth = "Earth"
    death_world = "Death World"


class Space(enum.Enum):
    pyrus = "Pyrus"
    terra = "Terra"
    aquas = "Aquas"
    darkus = "Darkus"
    Haos = "Haos"
    ventus = "Ventus"


association_table_battles = Table('association_battles', Base.metadata,
    Column("battle_id", ForeignKey("battles.id"), primary_key=True),
    Column("hero_id", ForeignKey("heroes.id"), primary_key=True),
    Column("hero_moto_id", ForeignKey("hero_motos.id"), primary_key=True)
)


class Heroes(Base):

    __tablename__ = "heroes"
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True, autoincrement=True)
    side = Column(Enum(Side))
    name = Column(String(40), nullable=False, unique=True)
    space = Column(Enum(Space))
    force = Column(Integer)
    moto = relationship("Hero_motos")
    battles = relationship("Battles", secondary=association_table_battles, back_populates="hero")

    def __str__(self) -> str:
        return f'Hero name: {self.name}, space: {self.space}, side: {self.side}, force = {self.force}G'


class Hero_motos(Base):

    __tablename__ = "hero_motos"
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True, autoincrement=True)
    hero_id = Column(Integer, ForeignKey("heroes.id", ondelete='CASCADE'))
    moto_id = Column(Integer)
    moto = Column(String(100))
    battles = relationship("Battles", secondary=association_table_battles, back_populates="moto")

    def __str__(self) -> str:
        return f"Hero's moto is {self.moto}"


class Battles(Base):

    __tablename__ = "battles"
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True, autoincrement=True)
    hero = relationship("Heroes",  secondary=association_table_battles, back_populates="battles")
    moto = relationship("Hero_motos",  secondary=association_table_battles, back_populates="battles")
    hero_1_id = Column(Integer, ForeignKey("heroes.id"))
    hero_1_moto_id = Column(Integer, ForeignKey("hero_motos.id"))
    hero_2_id = Column(Integer, ForeignKey("heroes.id"))
    hero_2_moto_id = Column(Integer, ForeignKey("hero_motos.id"))
    winner = Column(Integer)

    def __str__(self) -> str:
        battle_result = "Draw"
        if self.winner == 1:
            battle_result = f"Winner {self.hero_1_id}"
        elif self.winner == 2:
            battle_result = f"Winner {self.hero_2_id}"
        return f'{self.hero_1_id} vs {self.hero_2_id} | Result: {battle_result}'



class Hero_stories(Base):

    __tablename__ = "hero_stories"
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True, autoincrement=True)
    hero = relationship("Heroes", backref=backref("hero_stories", uselist=False))
    hero_id = Column(Integer, ForeignKey("heroes.id", ondelete='CASCADE'))
    story = Column(String(1000))

    def __str__(self):
        return f"Hero id: {self.hero_id}\n{self.story}"


Base.metadata.create_all(engine)

OperationalError: (psycopg2.OperationalError) could not translate host name "db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [3]:
Session = sessionmaker(bind=engine)

In [4]:
def confirm_input_value(field: str, value_field: str) -> bool:
    confirmation: str = input(f"So, {field} is {value_field}.\n Is that right? (Y/N)")

    return len(confirmation) > 0 and confirmation[0].lower() == 'y'


def input_side() -> Side:
    result_side: Side | None = None

    while not result_side:
        try:
            result_side = Side(input(f"Input one of sides: {[side.value for side in list(Side)]}\n"))
        except ValueError:
            print("Wrong side name")

    return result_side


def input_str(field: str) -> str:
    value_field: str = ''

    while value_field == '':
        value_field = input(f"Input {field}:\n")
        
        if not confirm_input_value(field, value_field):
            value_field = ''

    return value_field


def input_space() -> Space:
    result_space: Space | None = None

    while not result_space:
        try:
            result_space = Space(input(f"Input one of spaces: {[space.value for space in list(Space)]}\n"))
        except ValueError:
            print("Wrong space name")

    return result_space


def input_force() -> int:
    force: int = -1

    while force <= 0:
        try:
            force = int(input("Input hero's force value :\n"))

            if force <= 0:
                raise ValueError()
        except ValueError:
            print("Wrong force value. Value should be positive and more than 0\n")
        else:
            if not confirm_input_value("hero's force", str(force)):
                force = -1
        
    return force


def get_hero_by_name(heroes_query: list[Heroes]) -> Heroes | None:
    """
    Return a hero if there is a such hero's name in database. Else return None
    """
    print(f"Valid hero's names: {[hero.name for hero in heroes_query.all()]}")
    hero_name = input_str("hero's name")
    
    heroes: list[Heroes] = heroes_query.filter(Heroes.name == hero_name).all()

    if len(heroes) == 1:
        return heroes[0]

    print("Wrong hero's name")
    
    return None
    

def add_hero() -> None:
    name = input_str("hero's name")
    side = input_side()
    space = input_space()
    force = input_force()

    new_hero = Heroes(
        name=name, 
        side=side, 
        space=space, 
        force=force
    )

    with Session() as session:
        session.add(new_hero)
        session.commit()


def add_hero_moto() -> None:
    with Session() as session:
        heroes_query: list[Heroes] = session.query(Heroes)

    hero: Heroes | None = None
    
    while not hero:
        hero = get_hero_by_name(heroes_query)

    moto: str = input_str("hero's moto")

    with Session() as session:
        hero_motos_max_id: int | None = session.query(func.max(Hero_motos.moto_id)).filter(Hero_motos.hero_id == hero.id).first()[0]

    if hero_motos_max_id is None:
        hero_motos_max_id = 0

    new_hero_moto: Hero_motos = Hero_motos(
        hero_id=hero.id, 
        moto_id=hero_motos_max_id + 1, 
        moto=moto
    )

    with Session() as session:
        session.add(new_hero_moto)
        session.commit()


def add_hero_story() -> None:
    with Session() as session:
        heroes_query: list[Heroes] = session.query(Heroes)

    hero: Heroes | None = None
    
    while not hero:
        hero = get_hero_by_name(heroes_query)

    story: str = input_str("hero's story")

    new_hero_story: Hero_stories = Hero_stories(
        hero_id=hero.id,
        story=story
    )

    with Session() as session:
        session.add(new_hero_story)
        session.commit()


def delete_hero() -> None:
    with Session() as session:
        heroes_query: list[Heroes] = session.query(Heroes)

    hero_for_delete: Heroes | None = None
    
    while not hero:
        hero_for_delete = get_hero_by_name(heroes_query)
    
    with Session() as session:
        session.delete(hero_for_delete)
        session.commit()


In [5]:
dan = Heroes(id=1, side=Side.earth, name="Dan", space=Space.pyrus, force=890)
shun = Heroes(id=2, side=Side.earth, name="Shun", space=Space.ventus, force=920)
runo = Heroes(id=3, side=Side.earth, name="Runo", space=Space.Haos, force=850)
mask = Heroes(id=4, side=Side.death_world, name="The Mask", space=Space.darkus, force=950)
deisy = Heroes(id=5, side=Side.death_world, name="Deisy", space=Space.terra, force=830)
marucho_clone = Heroes(id=6, side=Side.death_world, name="Marucho clone", space=Space.aquas, force=870)

with Session() as session:
    session.add(dan)
    session.add(shun)
    session.add(runo)
    session.add(mask)
    session.add(deisy)
    session.add(marucho_clone)
    session.commit()



2022-03-28 22:49:42,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:49:42,494 INFO sqlalchemy.engine.Engine INSERT INTO heroes (id, side, name, space, force) VALUES (?, ?, ?, ?, ?)
2022-03-28 22:49:42,495 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ((1, 'earth', 'Dan', 'pyrus', 890), (2, 'earth', 'Shun', 'ventus', 920), (3, 'earth', 'Runo', 'Haos', 850), (4, 'death_world', 'The Mask', 'darkus', 950), (5, 'death_world', 'Deisy', 'terra', 830), (6, 'death_world', 'Marucho clone', 'aquas', 870))
2022-03-28 22:49:42,497 INFO sqlalchemy.engine.Engine COMMIT


C:\Users\piros\AppData\Local\Temp\ipykernel_14588\2588655943.py:1: SAWarning: relationship 'Hero_motos.battles' will copy column battles.id to column association_battles.battle_id, which conflicts with relationship(s): 'Heroes.battles' (copies battles.id to association_battles.battle_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="battles"' to the 'Hero_motos.battles' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  dan = Heroes(id=1, side=Side.earth, name="Dan", space=Space.pyrus, force=890)
C:\Users\piros\AppData\Local\Temp\ipykernel_14588\2588655943.py:1: SAWarning: relationship 'Battles.hero' wi

In [6]:
dan_moto_1 = Hero_motos(id=1, hero_id=1, moto_id=1, moto="Drago is on the field")
dan_moto_2 = Hero_motos(id=2, hero_id=1, moto_id=2, moto="Saurus appears")
shun_moto_1 = Hero_motos(id=3, hero_id=2, moto_id=1, moto="Skyress is on the field")
shun_moto_2 = Hero_motos(id=4, hero_id=2, moto_id=2, moto="Highlash join")
runo_moto_1 = Hero_motos(id=5, hero_id=3, moto_id=1, moto="Tigrerra is on the field")
runo_moto_2 = Hero_motos(id=6, hero_id=3, moto_id=2, moto="Open the gate card")
mask_moto_1 = Hero_motos(id=7, hero_id=4, moto_id=1, moto="Hydronoid is on the field")
mask_moto_2 = Hero_motos(id=8, hero_id=4, moto_id=2, moto="Reaper reapers")
deisy_moto_1 = Hero_motos(id=9, hero_id=5, moto_id=1, moto="Claive is on the field")
deisy_moto_2 = Hero_motos(id=10, hero_id=5, moto_id=2, moto="The force of terra")
marucho_clone_moto_1 = Hero_motos(id=11, hero_id=6, moto_id=1, moto="Froggy is on the field")
marucho_clone_moto_2 = Hero_motos(id=12, hero_id=6, moto_id=2, moto="Tsunami")

with Session() as session:
    session.add(dan_moto_1)
    session.add(dan_moto_2)
    session.add(shun_moto_1)
    session.add(shun_moto_2)
    session.add(runo_moto_1)
    session.add(runo_moto_2)
    session.add(mask_moto_1)
    session.add(mask_moto_2)
    session.add(deisy_moto_1)
    session.add(deisy_moto_2)
    session.add(marucho_clone_moto_1)
    session.add(marucho_clone_moto_2)
    session.commit()


2022-03-28 22:49:47,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:49:47,538 INFO sqlalchemy.engine.Engine INSERT INTO hero_motos (id, hero_id, moto_id, moto) VALUES (?, ?, ?, ?)
2022-03-28 22:49:47,539 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ((1, 1, 1, 'Drago is on the field'), (2, 1, 2, 'Saurus appears'), (3, 2, 1, 'Skyress is on the field'), (4, 2, 2, 'Highlash join'), (5, 3, 1, 'Tigrerra is on the field'), (6, 3, 2, 'Open the gate card'), (7, 4, 1, 'Hydronoid is on the field'), (8, 4, 2, 'Reaper reapers')  ... displaying 10 of 12 total bound parameter sets ...  (11, 6, 1, 'Froggy is on the field'), (12, 6, 2, 'Tsunami'))
2022-03-28 22:49:47,541 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
dan_story = Hero_stories(id=1, hero_id=1, story="Dan is the main human protagonist in the series, who, is 12 years old (11 in the Japanese dub) in the first season, 15 in New Vestroia, 16 in Gundalian Invaders, and 17 in Mechtanium Surge. In the beginning he is ranked #121, and by episode 39, he is ranked #1, and has become ranked #4 or under in episode 6 of Mechtanium Surge.")
shun_story = Hero_stories(id=2, hero_id=2, story="Shun is 13 years old (11 in the Japanese dub), has black hair, which is tied into a long ponytail, bronze-brown eyes and wears a purple jacket over a dark, dark blue (almost black) sleeveless shirt. He is a master of Bakugan and co-created the rules with Dan. He is also Dan's childhood best friend. Shun is a loner, a boy of very few words, but yet is willing to help his friends at almost every turn. He is a Ventus brawler and he approaches Bakugan like a ninja.")
runo_story = Hero_stories(id=3, hero_id=3, story="Runo is a 12-year-old (11 in the Japanese dub) girl who loves playing Bakugan against skilled people so she can show off. Runo is a Haos brawler. Her Guardian Bakugan is a Haos Tigrerra (the others being Terrowclaw and Saurus), who is very obedient and powerful in battle.")
mask_story = Hero_stories(id=4, hero_id=4, story="Alice(The Mask) is a 14-year-old girl from Moscow, Russia. Although she knows almost everything about the game, she rarely plays it; she is generally afraid of hurting others, including Bakugan. She usually just gives advice to the other brawlers, which she prefers over battling. Alice is kind and caring, worried forand compassionate towards others and does not think of herself. She also adores Shun. Alice was sometimes unconsciously possessed by 'Masquerade' (her mysterious alter-ego)")
deisy_story = Hero_stories(id=5, hero_id=5, story="Older sister of Julie")
marucho_clone_story = Hero_stories(id=6, hero_id=6, story="Marucho loves to play Bakugan and is also able to change attributes. Marucho is eleven (ten in the Japanese dub) and is an aquas brawler. He is seen after being defeated by Volt in episode 13, being held as a hostage in episodes 15 and 17, and was later freed by Mira and Spectra in episode 20, even though he is unaware of this.")

with Session() as session:
    session.add(dan_story)
    session.add(shun_story)
    session.add(runo_story)
    session.add(mask_story)
    session.add(deisy_story)
    session.add(marucho_clone_story)
    session.commit()


2022-03-28 22:49:50,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:49:50,274 INFO sqlalchemy.engine.Engine INSERT INTO hero_stories (id, hero_id, story) VALUES (?, ?, ?)
2022-03-28 22:49:50,274 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ((1, 1, 'Dan is the main human protagonist in the series, who, is 12 years old (11 in the Japanese dub) in the first season, 15 in New Vestroia, 16 in Gundali ... (31 characters truncated) ... um Surge. In the beginning he is ranked #121, and by episode 39, he is ranked #1, and has become ranked #4 or under in episode 6 of Mechtanium Surge.'), (2, 2, "Shun is 13 years old (11 in the Japanese dub), has black hair, which is tied into a long ponytail, bronze-brown eyes and wears a purple jacket over a ... (167 characters truncated) ...  a boy of very few words, but yet is willing to help his friends at almost every turn. He is a Ventus brawler and he approaches Bakugan like a ninja."), (3, 3, 'Runo is a 12-year-old (11 in the J

In [8]:
import random
from math import exp


def randomize_number(limit: int) -> int:
    return int(limit*random.random())


def choose_side(without_sides: set[Side]) -> Side:

    actual_sides: list[Side] = [side for side in list(Side) if side not in without_sides]
    number_sides: int = len(actual_sides)

    index_side: int = randomize_number(number_sides)
    
    return actual_sides[index_side]


def choose_hero(side: Side) -> Heroes:
    with Session() as session:
        heroes: list[Heroes] = session.query(Heroes).filter(Heroes.side == side).all()

    number_heroes: int = len(heroes)

    index_hero: int = randomize_number(number_heroes)

    return heroes[index_hero]


def choose_moto(hero: Heroes) -> Hero_motos:
    with Session() as session:
        motos: list[Hero_motos] = session.query(Hero_motos).filter(Hero_motos.hero_id == hero.id).all()

    number_motos: int = len(motos)

    index_moto: int = randomize_number(number_motos)

    return motos[index_moto]


def count_puasson_distibution(coef_lambda: float) -> float:
    return exp(-coef_lambda) * coef_lambda


def run_battle() -> None:
    first_side: Side = choose_side(set())
    second_side: Side = choose_side({first_side})

    first_hero: Heroes = choose_hero(first_side)
    second_hero: Heroes = choose_hero(second_side)

    first_hero_moto: Hero_motos = choose_moto(first_hero)
    second_hero_moto: Hero_motos = choose_moto(second_hero)

    draw_probability = count_puasson_distibution(first_hero.force / second_hero.force)
    first_win_probability = (1 - draw_probability) * first_hero.force / (first_hero.force + second_hero.force)
    second_win_probability = 1 - draw_probability - first_win_probability

    result_probability = random.random()
    winner: int = 0

    if first_win_probability > result_probability:
        winner = 1
    elif first_win_probability + second_win_probability > result_probability:
        winner = 2

    battle = Battles(
        hero_1_id=first_hero.id,
        hero_1_moto_id=first_hero_moto.id,
        hero_2_id=second_hero.id, 
        hero_2_moto_id=second_hero_moto.id, 
        winner=winner
    )

    with Session() as session:
        session.add(battle)
        session.commit()



In [9]:
for _ in range(20):
    run_battle()

2022-03-28 22:49:56,873 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:49:56,876 INFO sqlalchemy.engine.Engine SELECT heroes.id AS heroes_id, heroes.side AS heroes_side, heroes.name AS heroes_name, heroes.space AS heroes_space, heroes.force AS heroes_force 
FROM heroes 
WHERE heroes.side = ?
2022-03-28 22:49:56,877 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ('death_world',)
2022-03-28 22:49:56,878 INFO sqlalchemy.engine.Engine ROLLBACK
2022-03-28 22:49:56,879 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:49:56,880 INFO sqlalchemy.engine.Engine SELECT heroes.id AS heroes_id, heroes.side AS heroes_side, heroes.name AS heroes_name, heroes.space AS heroes_space, heroes.force AS heroes_force 
FROM heroes 
WHERE heroes.side = ?
2022-03-28 22:49:56,880 INFO sqlalchemy.engine.Engine [cached since 0.004591s ago] ('earth',)
2022-03-28 22:49:56,881 INFO sqlalchemy.engine.Engine ROLLBACK
2022-03-28 22:49:56,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)


In [11]:
with Session() as session:
    for battle in session.query(Battles).all():
        print(battle)


2 vs 6 | Result: Winner 2
2 vs 4 | Result: Winner 2
1 vs 5 | Result: Winner 5
1 vs 5 | Result: Winner 1
6 vs 3 | Result: Winner 6
3 vs 6 | Result: Winner 6
3 vs 6 | Result: Winner 6
3 vs 5 | Result: Draw
1 vs 4 | Result: Draw
2 vs 6 | Result: Draw
3 vs 4 | Result: Draw
3 vs 4 | Result: Winner 3
5 vs 1 | Result: Draw
6 vs 1 | Result: Winner 1
6 vs 3 | Result: Draw
3 vs 5 | Result: Winner 5
6 vs 1 | Result: Winner 1
3 vs 6 | Result: Winner 3
1 vs 6 | Result: Winner 6
5 vs 3 | Result: Winner 3


In [ ]:
with Session() as session:
        hero: Heroes = session.query(func.max(Hero_motos.moto_id)).filter(Hero_motos.hero_id == 3).all()

print(hero[0][0])

2


In [13]:
with Session() as session:
    heroes = session.query(Heroes).all()

print(heroes)

2022-03-28 22:53:05,818 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:53:05,819 INFO sqlalchemy.engine.Engine SELECT heroes.id AS heroes_id, heroes.side AS heroes_side, heroes.name AS heroes_name, heroes.space AS heroes_space, heroes.force AS heroes_force 
FROM heroes
2022-03-28 22:53:05,821 INFO sqlalchemy.engine.Engine [cached since 129.4s ago] ()
2022-03-28 22:53:05,823 INFO sqlalchemy.engine.Engine ROLLBACK
[<__main__.Heroes object at 0x000002DC68A21360>, <__main__.Heroes object at 0x000002DC68A213F0>, <__main__.Heroes object at 0x000002DC68A21660>, <__main__.Heroes object at 0x000002DC68A214B0>, <__main__.Heroes object at 0x000002DC68A216F0>, <__main__.Heroes object at 0x000002DC68A21780>, <__main__.Heroes object at 0x000002DC68A218A0>]


In [10]:
add_hero()

2022-03-28 22:50:50,302 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:50:50,303 INFO sqlalchemy.engine.Engine INSERT INTO heroes (side, name, space, force) VALUES (?, ?, ?, ?)
2022-03-28 22:50:50,303 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('earth', 'Julie', 'terra', 850)
2022-03-28 22:50:50,315 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
add_hero_moto()

Valid hero's names: ['Dan', 'Shun', 'Runo', 'The Mask', 'Deisy', 'Marucho clone', 'Julie']


In [14]:
add_hero_story()

Valid hero's names: ['Dan', 'Shun', 'Runo', 'The Mask', 'Deisy', 'Marucho clone', 'Julie']


In [11]:
delete_hero()

2022-03-28 22:50:56,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-28 22:50:56,456 INFO sqlalchemy.engine.Engine SELECT heroes.id AS heroes_id, heroes.side AS heroes_side, heroes.name AS heroes_name, heroes.space AS heroes_space, heroes.force AS heroes_force 
FROM heroes
2022-03-28 22:50:56,456 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ()
Valid hero's names: ['Dan', 'Shun', 'Runo', 'The Mask', 'Deisy', 'Marucho clone', 'Julie']
2022-03-28 22:51:00,576 INFO sqlalchemy.engine.Engine SELECT heroes.id AS heroes_id, heroes.side AS heroes_side, heroes.name AS heroes_name, heroes.space AS heroes_space, heroes.force AS heroes_force 
FROM heroes 
WHERE heroes.name = ?
2022-03-28 22:51:00,576 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ('Julie',)


InvalidRequestError: Object '<Heroes at 0x2dc67d81360>' is already attached to session '105' (this is '106')

In [ ]:
with Session() as session:
    hero_motos_max_id: int = session.query(func.max(Hero_motos.moto_id)).filter(Hero_motos.hero_id == 7).first()

print(hero_motos_max_id[0])

1
